In [3]:
import json
import re
import fitz
from datetime import datetime, timezone
import pandas as pd
import os

In [4]:
from pathlib import Path

def finde_alle_pdfs(root_ordner):
    return [str(p) for p in Path(root_ordner).rglob('*.pdf')]

main_folder = os.path.join("..", "sample_roster", "bbtc_pl_2025")
test_roster_paths = finde_alle_pdfs(main_folder)

In [5]:
test_roster_paths[0].split("\\")

['..', 'sample_roster', 'bbtc_pl_2025', 'High Elfs - Akorus.pdf']

In [6]:
# Python
import fitz  # PyMuPDF

# Open the PDF
doc = fitz.open(test_roster_paths[4])

team_data = {}

extraction_step = 'Race'
for page_number, page in enumerate(doc, start=1):
    # Extract text in blocks/spans with details
    blocks = page.get_text("dict")["blocks"]
    
    for block in blocks:
        if "lines" in block:
            for line in block["lines"]:
                for span in line["spans"]:
                    text = span["text"]
                    color = span["color"]  # RGB, float 0-1 representation
                    # print(f"Page {page_number}: '{text}' Color: {color}")
                    
                    if extraction_step == 'Race':
                        if text == 'COACH NAME':
                            extraction_step = 'Coach'
                        if 'Race' in team_data:
                            team_data


In [44]:
# Python
import fitz  # PyMuPDF

def process_team_pdf(roster_path):
    # Open the PDF
    doc = fitz.open(roster_path)

    roster_path_split = roster_path.split("\\")
    pdf_name = roster_path_split[-1]
    
    team_data = {
        'PDF Name': pdf_name
    }
    
    extraction_step = 'Race'
    for page_number, page in enumerate(doc, start=1):
        # Extract text in blocks/spans with details
        blocks = page.get_text("dict")["blocks"]

        for block in blocks:
            if "lines" in block:
                for line in block["lines"]:
                    for span in line["spans"]:
                        text = span["text"]
                        color = span["color"]  # RGB, float 0-1 representation
                        # print(f"Page {page_number}: '{text}' Color: {color}")
                        # Adjust extraction step
                        if text == 'SIDELINE':
                            extraction_step = 'Sideline'
                            sideline_ctr = 0
                        elif text == 'INDUCEMENTS':
                            extraction_step = 'Inducements'
                            next_name = None
                        elif text == 'SUMMARY':
                            summary_ctr = 0
                            extraction_step = 'Summary'
                            summary_ctr = -1
    
                        if extraction_step == 'Race':
                            if text == 'COACH NAME':
                                extraction_step = 'Coach'
                                continue
                            if 'Race' in team_data:
                                team_data['Race'] += ' ' + text
                            else:
                                team_data['Race'] = text

                        elif extraction_step == 'Coach':
                            team_data['Coach'] = text
                            extraction_step = 'Team'

                        elif extraction_step == 'Team':
                            if text == 'TEAM NAME':
                                continue
                            elif text == 'SIDELINE':
                                extraction_step = 'Sideline'
                                sideline_ctr = 0
                                
                            elif 'Team' in team_data:
                                team_data['Team'] += ' ' + text
                            else:
                                team_data['Team'] = text

                        elif extraction_step == 'Sideline':
                            sideline_properties = [
                                'Apothecary',
                                'Assistant coaches',
                                'Cheerleaders',
                                'Dedicated fans',
                                'Re-rolls',
                            ]
                            #print('START', sideline_ctr, text)
                            if sideline_ctr >= len(sideline_properties):
                                extraction_step = 'Inducements'
                                continue
                            next_sideline = sideline_properties[sideline_ctr]
                            # print(sideline_ctr, next_sideline, text)
                            if text == 'SIDELINE':
                                continue
                            elif text in sideline_properties:
                                if (sideline_ctr == 0) and (text != 'Apothecary'):
                                    sideline_ctr += 1
                                    # print("BUMP COUNTER")
                                # print('text in properties', text)
                                continue
                            else:
                                # print('SAVE', next_sideline, text)
                                team_data[f'Sideline - {next_sideline}'] = text
                                sideline_ctr += 1

                        elif extraction_step == 'Inducements':
                            # print(f"INDUCEMENTS: {text}")
                            if text in ['SUMMARY', 'No inducements', 'LEAGUES & SPECIAL']:
                                summary_ctr = 0
                                extraction_step = 'Summary'
                                summary_ctr = -1
                                continue
                            if text == 'INDUCEMENTS':
                                continue
                            if next_name is None:
                                next_name = text
                            else:
                                team_data[f'Inducement - {next_name}'] = text
                                next_name = None

                        elif extraction_step == 'Summary':
                            print(f"SUMMARY {summary_ctr} | {text}")
                            summary_steps = [
                                'Players cost',
                                None,
                                'Skills cost',
                                None,
                                'Inducement cost',
                                None,
                                'Sideline cost',
                                None,
                                'Primary skills',
                                None,
                                'Secondary skills',
                                None
                            ]
                            if summary_ctr == len(summary_steps):
                                extraction_step = 'Players'
                                player_ctr = -1
                                continue

                            if text == 'SUMMARY':
                                summary_ctr = 0
                                continue
                            elif summary_ctr == -1:
                                continue

                            if (summary_ctr % 2) == 1:
                                team_data[f'Summary - {summary_steps[summary_ctr - 1]}'] = text
                            summary_ctr += 1
    
                        elif extraction_step == 'Players':
                            if text == 'COST':
                                team_data['Players'] = []
                                player_ctr = 1
                                next_player_property = 'Name'
                                current_player = {
                                    'ctr': player_ctr,
                                    'position_name': None,
                                    'primary_1': None,
                                    'primary_2': None,
                                    'secondary_1': None,
                                    'secondary_2': None,
                                    'star': False,
                                }
                                continue
                            elif player_ctr == -1:
                                continue

                            if re.fullmatch(r"\b\d+k\b", text):
                                team_data['Players'].append(current_player)
                                player_ctr += 1
                                current_player = {
                                    'ctr': player_ctr,
                                    'position_name': None,
                                    'primary_1': None,
                                    'primary_2': None,
                                    'secondary_1': None,
                                    'secondary_2': None,
                                    'star': False,
                                }
                                next_player_property = 'Name'

                            elif next_player_property == 'Name':
                                current_player['position_name'] = " ".join(text.split()[1:])
                                next_player_property = 'Skills'

                            elif next_player_property == 'Skills':
                                if color == 681912:
                                    if current_player['primary_1'] is None:
                                        current_player['primary_1'] = text.strip().strip(',')
                                    elif current_player['primary_2'] is None:
                                        current_player['primary_2'] = text.strip().strip(',')
                                    else:
                                        raise RuntimeError(f'Unexpected Skill | Color: {color} - Text: {text}')
                                if color == 4822027:
                                    if current_player['secondary_1'] is None:
                                        current_player['secondary_1'] = text.strip().strip(',')
                                    elif current_player['secondary_2'] is None:
                                        current_player['secondary_2'] = text.strip().strip(',')
                                    else:
                                        raise RuntimeError(f'Unexpected Skill | Color: {color} - Text: {text}')
                            
                            if text == 'Special skill: ':
                                current_player['star'] = True
                                      
    return team_data
                                    
                                
                            
processed_roster = []
for roster_path in test_roster_paths:
    team_data = process_team_pdf(roster_path)
    processed_roster.append(team_data)
    # print(json.dumps(team_data, indent=4))

SUMMARY -1 | LEAGUES & SPECIAL
SUMMARY -1 | RULES
SUMMARY -1 | • Elven Kingdom League
SUMMARY -1 | SUMMARY
SUMMARY 0 | Players cost
SUMMARY 1 | 955k
SUMMARY 2 | Skills cost
SUMMARY 3 | 200k
SUMMARY 4 | Inducement cost
SUMMARY 5 | 0k
SUMMARY 6 | Sideline cost
SUMMARY 7 | 210k
SUMMARY 8 | Primary skills
SUMMARY 9 | 7
SUMMARY 10 | Secondary skills
SUMMARY 11 | 0
SUMMARY 12 | # POSITION
SUMMARY -1 | RULES
SUMMARY -1 | • Brawlin' Brutes
SUMMARY -1 | • Favoured of Khorne
SUMMARY -1 | • Chaos Clash
SUMMARY -1 | SUMMARY
SUMMARY 0 | Players cost
SUMMARY 1 | 970k
SUMMARY 2 | Skills cost
SUMMARY 3 | 230k
SUMMARY 4 | Inducement cost
SUMMARY 5 | 25k
SUMMARY 6 | Sideline cost
SUMMARY 7 | 180k
SUMMARY 8 | Primary skills
SUMMARY 9 | 6
SUMMARY 10 | Secondary skills
SUMMARY 11 | 1
SUMMARY 12 | # POSITION
SUMMARY -1 | LEAGUES & SPECIAL
SUMMARY -1 | RULES
SUMMARY -1 | • Sylvanian Spotlight
SUMMARY -1 | • Masters of Undeath
SUMMARY -1 | SUMMARY
SUMMARY 0 | Players cost
SUMMARY 1 | 940k
SUMMARY 2 | Skills c

In [45]:
len(processed_roster)

5

In [46]:
2 % 2

0

In [48]:
processed_roster[1]

{'PDF Name': 'Khorne - Akorus.pdf',
 'Race': 'Khorne',
 'Coach': 'Akorus',
 'Team': 'Emerald - Khorne',
 'Sideline - Apothecary': 'Yes',
 'Sideline - Assistant coaches': '1',
 'Sideline - Cheerleaders': '0',
 'Sideline - Dedicated fans': '0',
 'Sideline - Re-rolls': '2',
 'Inducement - Team Mascot': '1',
 'Summary - Players cost': '970k',
 'Summary - Skills cost': '230k',
 'Summary - Inducement cost': '25k',
 'Summary - Sideline cost': '180k',
 'Summary - Primary skills': '6',
 'Summary - Secondary skills': '1',
 'Players': [{'ctr': 1,
   'position_name': 'Bloodspawn',
   'primary_1': None,
   'primary_2': None,
   'secondary_1': 'Block',
   'secondary_2': None,
   'star': False},
  {'ctr': 2,
   'position_name': 'Bloodseeker',
   'primary_1': 'Block',
   'primary_2': None,
   'secondary_1': None,
   'secondary_2': None,
   'star': False},
  {'ctr': 3,
   'position_name': 'Bloodseeker',
   'primary_1': 'Block',
   'primary_2': None,
   'secondary_1': None,
   'secondary_2': None,
   's

In [49]:
rows = []
error_rows = []
for team_data in processed_roster:
    if 'Players' in team_data:
        for player in team_data['Players']:
            prefix = f'player_{player["ctr"]}'
            for key, value in player.items():
                if key == 'ctr':
                    continue
                
                team_data[f'{prefix}_{key}'] = value
        
        del team_data['Players']
        rows.append(team_data)
    elif 'player_11_star' in team_data:
        rows.append(team_data)
    else:
        error_rows.append(team_data)

print('Rows', len(rows))
print('Errors', len(error_rows))

Rows 5
Errors 0


In [50]:
df = pd.DataFrame(rows)
df.head(5)

,PDF Name,Race,Coach,Team,Sideline - Apothecary,Sideline - Assistant coaches,Sideline - Cheerleaders,Sideline - Dedicated fans,Sideline - Re-rolls,Summary - Players cost,...,player_12_primary_2,player_12_secondary_1,player_12_secondary_2,player_12_star,player_13_position_name,player_13_primary_1,player_13_primary_2,player_13_secondary_1,player_13_secondary_2,player_13_star
0,High Elfs - Akorus.pdf,High Elf,Akorus,Emerald - High Elfs,Yes,1,0,0,3,955k,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Khorne - Akorus.pdf,Khorne,Akorus,Emerald - Khorne,Yes,1,0,0,2,970k,...,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN
2,Necro - Schlachtenlenker.pdf,Necromantic Horror,Schlachtenlenker,Emerald - Necro,NaN,0,0,0,3,940k,...,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN
3,Nurgle - Akorus.pdf,Nurgle,Akorus,Emerald - Nurgle,NaN,0,0,0,3,960k,...,NaN,NaN,NaN,False,Rotter Lineman,NaN,NaN,NaN,NaN,False
4,Vamps - Akorus.pdf,Vampire,Akorus,Emerald - Vamps,No,0,0,0,4,920k,...,NaN,NaN,NaN,False,Thrall Lineman,NaN,NaN,NaN,NaN,False


In [61]:
df_emerald_bowl_processing = df[['PDF Name', 'Race', 'Coach', 'Team',
       'Summary - Players cost', 'Summary - Skills cost',
       'Summary - Inducement cost', 'Summary - Sideline cost',
       'Summary - Primary skills', 'Summary - Secondary skills']]
df_emerald_bowl_processing['emerald_bowl_skill_cost'] = df_emerald_bowl_processing['Summary - Primary skills'].astype(int) * 20  + df_emerald_bowl_processing['Summary - Secondary skills'].astype(int) * 30
df_emerald_bowl_processing

C:\Users\maxph\AppData\Local\Temp\ipykernel_39844\1391038457.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emerald_bowl_processing['emerald_bowl_skill_cost'] = df_emerald_bowl_processing['Summary - Primary skills'].astype(int) * 20  + df_emerald_bowl_processing['Summary - Secondary skills'].astype(int) * 30


,PDF Name,Race,Coach,Team,Summary - Players cost,Summary - Skills cost,Summary - Inducement cost,Summary - Sideline cost,Summary - Primary skills,Summary - Secondary skills,emerald_bowl_skill_cost
0,High Elfs - Akorus.pdf,High Elf,Akorus,Emerald - High Elfs,955k,200k,0k,210k,7,0,140
1,Khorne - Akorus.pdf,Khorne,Akorus,Emerald - Khorne,970k,230k,25k,180k,6,1,150
2,Necro - Schlachtenlenker.pdf,Necromantic Horror,Schlachtenlenker,Emerald - Necro,940k,180k,0k,210k,6,0,120
3,Nurgle - Akorus.pdf,Nurgle,Akorus,Emerald - Nurgle,960k,210k,25k,180k,7,0,140
4,Vamps - Akorus.pdf,Vampire,Akorus,Emerald - Vamps,920k,150k,25k,240k,6,0,120


In [60]:
now_utc = datetime.now(timezone.utc)
timestamp_utc_str = now_utc.strftime("%Y%m%dT%H%M%S")
df.to_csv(f'roster_extracted_{timestamp_utc_str}.csv')

In [13]:
now_utc = datetime.now(timezone.utc)
timestamp_utc_str = now_utc.strftime("%Y%m%dT%H%M%S")
df_update.to_csv(f'eb_roster_update_{timestamp_utc_str}.csv')

In [20]:
df_error = pd.DataFrame(error_rows)
now_utc = datetime.now(timezone.utc)
timestamp_utc_str = now_utc.strftime("%Y%m%dT%H%M%S")
df_error = pd.DataFrame(error_rows).to_csv(f'eb_roster_errors_{timestamp_utc_str}.csv')
len(error_rows)

17

In [17]:
df_error[["Event Team Name", "Event Type",  "PDF Name"]].head(30)

TypeError: 'NoneType' object is not subscriptable

In [69]:
error_rows

[{'Event Team Name': 'Florida Blood Bowl',
  'Event Type': 'Europen',
  'PDF Name': 'BB Roster - Eurobowl - Nurgle - Nurgle.pdf',
  'Race': 'Eurobowl - Nurgle Nurgle Team 11s Exhibition # NAME POSITION MA ST AG PA AV SKILLS HIRING FEE SPP 1 Player Name Bloater 4 4 4+ 6+ 10+ Disturbing Presence, Foul Appearance, Plague Ridden, Regeneration,  115,000 -6 2 Player Name Bloater 4 4 4+ 6+ 10+ Disturbing Presence, Foul Appearance, Plague Ridden, Regeneration,  115,000 -6 3 Player Name Bloater 4 4 4+ 6+ 10+ Disturbing Presence, Foul Appearance, Plague Ridden, Regeneration,  115,000 -6 4 Player Name Bloater 4 4 4+ 6+ 10+ Disturbing Presence, Foul Appearance, Plague Ridden, Regeneration,  115,000 -6 5 Player Name Pestigor 6 3 3+ 4+ 9+ Horns, Plague Ridden, Regeneration,  75,000 -14 6 Player Name Pestigor 6 3 3+ 4+ 9+ Horns, Plague Ridden, Regeneration,  75,000 -6 7 Player Name Pestigor 6 3 3+ 4+ 9+ Horns, Plague Ridden, Regeneration,  75,000 -6 8 Player Name Rotter Lineman 5 3 4+ 6+ 9+ Decay, Pl